In [1]:
#GAN architecture
import tensorflow as tf
from tensorflow import keras as ke
LR_shape = (32,32,3)
s = 2
HR_shape = (LR_shape[0]*s,LR_shape[1]*s,3)


def Generator(LR_shape = LR_shape,s = s):
    model = ke.Sequential()
    # model.add(ke.layers.Conv2D(3,(64,64),strides=(4,4),input_shape=LR_shape,activation='relu'))
    # model.add(ke.layers.MaxPool2D())
    model.add(ke.layers.Flatten(input_shape=(LR_shape)))
    model.add(ke.layers.Dense(LR_shape[0]*s*LR_shape[1]*s*3,activation='sigmoid'))
    HR_shape = (LR_shape[0]*s,LR_shape[1]*s,3)
    model.add(ke.layers.Reshape(HR_shape))
    return model

def Discriminator(HR_shape = (LR_shape[0]*s,LR_shape[1]*s,3)):
    model = ke.Sequential()
    # model.add(ke.layers.Conv2D(3,(64,64),input_shape = (1024,1024,3),activation='relu'))
    # model.add(ke.layers.MaxPool2D())
    model.add(ke.layers.Flatten(input_shape=HR_shape))
    model.add(ke.layers.Dense(500,activation='relu'))
    model.add(ke.layers.Dense(100,activation='relu'))
    model.add(ke.layers.Dense(1,activation='sigmoid'))
    return model

In [2]:
# drive access
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [3]:
# making low res images
import numpy as np
from PIL import Image
import numpy

def get_img_arr(path: str):
    img = np.array(Image.open(path))
    return img

def high_to_lowres(img_num: str):
    img_path = "/content/gdrive/MyDrive/00000/"+img_num+".png" #replace with location of data
    img_arr = get_img_arr(img_path)
    img_arr = img_arr.astype(np.float64)
    lr_img_arr = np.zeros((512,512,3))
    for k in range(lr_img_arr.shape[2]):
        for i in range(lr_img_arr.shape[0]):
            for j in range(lr_img_arr.shape[1]):
                lr_img_arr[i,j,k] = (img_arr[2*i,2*j,k] + img_arr[2*i,2*j+1,k] + img_arr[2*i+1,2*j,k] + img_arr[2*i+1,2*j+1,k])/4
    lr_img = Image.fromarray(lr_img_arr.astype(np.uint8))
    lr_img.save("/content/gdrive/MyDrive/lr_imgs/"+img_num+"_lr.png") #replace with desired location of LR images

img_num = '00000'
img0 = get_img_arr("/content/gdrive/MyDrive/00000/"+img_num+".png") #replace with location of data
img0.shape

(1024, 1024, 3)

In [ ]:
# apply above high_to_lowres function on '00000' to '01000'

for i in range(0,1000):
    img_num = str(i).zfill(5)
    high_to_lowres(img_num)


In [5]:
# model creation

combined_gan = ke.Sequential()

gen = Generator()
gen.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
combined_gan.add(gen)
disc = Discriminator()
disc.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
disc.trainable = False
combined_gan.add(disc)
combined_gan.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [6]:
disc.summary()
gen.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 12288)             0         
                                                                 
 dense_5 (Dense)             (None, 500)               6144500   
                                                                 
 dense_6 (Dense)             (None, 100)               50100     
                                                                 
 dense_7 (Dense)             (None, 1)                 101       
                                                                 
Total params: 6194701 (23.63 MB)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 6194701 (23.63 MB)
_________________________________________________________________
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Par

In [ ]:
# create a random array with number from 0 to 255 of shape (1024,1024,3) and forward propogate throught disc

rand_arr = np.random.randint(0,1,(64,64,3))
# rand_arr = rand_arr.reshape((11024,1024))
disc.predict(np.array([rand_arr]))


1/1 [==============================] - 4s 4s/step


array([[0.5]], dtype=float32)

In [7]:
import random as rd
import time

def pick_box(img:np.ndarray,box_size:int):
    img_shape = img.shape
    x_max, y_max = img_shape[0] - box_size, img_shape[1] - box_size
    x, y = rd.randint(0,x_max), rd.randint(0,y_max)
    box = img[x:x+box_size,y:y+box_size,:] #check shape of box
    assert box.shape == (box_size,box_size,3)
    return box

def make_real_batch_arr(img_names:list):
    boxes_lst = []
    for img_num in img_names:
        img_hr = get_img_arr("/content/gdrive/MyDrive/00000/"+img_num+".png") #replace with location of data
        box = pick_box(img_hr,HR_shape[0])
        boxes_lst.append(box)
    boxes_arr = np.array(boxes_lst)
    return boxes_arr/255

def gen_input_batch_arr(img_names:list):
    boxes_lst = []
    for img_num in img_names:
        img_lr = get_img_arr("/content/gdrive/MyDrive/lr_imgs/"+img_num+"_lr.png") #replace with location of LR imgs
        box = pick_box(img_lr,LR_shape[0])
        boxes_lst.append(box)
    boxes_arr = np.array(boxes_lst)
    return boxes_arr/255

def make_name_batches(img_names:list,batch_size:int):
    batches = []
    for i in range(len(img_names)//batch_size):
        batches.append(img_names[i*batch_size:(i+1)*batch_size])
    return batches


# arr = make_real_batch_arr(['00000','00001','00002'])
# arr.shape
# box = pick_box(img0,32)
# box.shape


In [11]:
start = time.time()

no_of_epochs = 10
batch_size = 100

real = np.ones((batch_size,1))
fake = np.zeros((batch_size,1))


img_names = [str(i).zfill(5) for i in range(900)]

for epoch in range(1,no_of_epochs+1):
    print("Epoch: ",epoch)
    rd.shuffle(img_names)
    batches = make_name_batches(img_names,batch_size)
    for batch in batches:
        real_batch_arr = make_real_batch_arr(batch)
        gen_input_arr = gen_input_batch_arr(batch)
        fake_batch_arr = gen.predict(gen_input_arr)
        #disc training
        weights = disc.get_weights()[0]
        disc.train_on_batch(real_batch_arr,real)
        disc.train_on_batch(fake_batch_arr,fake)
        assert np.any(disc.get_weights()[0] != weights)

        #combined gan training
        combined_gan.train_on_batch(gen_input_arr,real)

end = time.time()
print("Time Taken: ",end-start)
combined_gan.save("/content/gdrive/MyDrive/model/combined_gan") # replace with desired location to save model
gen.save("/content/gdrive/MyDrive/model/generator") #replace with desired location to save model
disc.save("/content/gdrive/MyDrive/model/discriminator") #replace with desired location to save model


Epoch:  1
4/4 [==============================] - 0s 31ms/step
Epoch:  2
4/4 [==============================] - 0s 34ms/step
Epoch:  3
4/4 [==============================] - 0s 33ms/step
Epoch:  4
4/4 [==============================] - 0s 32ms/step
Epoch:  5
4/4 [==============================] - 0s 47ms/step
Epoch:  6
4/4 [==============================] - 0s 34ms/step
Epoch:  7
4/4 [==============================] - 0s 32ms/step
Epoch:  8
4/4 [==============================] - 0s 47ms/step
Epoch:  9
4/4 [==============================] - 0s 32ms/step
Epoch:  10
4/4 [==============================] - 0s 46ms/step
Time Taken:  516.0041506290436


In [ ]:
#get the saved models back as the same variable name

combined_gan = ke.models.load_model("/content/gdrive/MyDrive/model/combined_gan") #replace with location of model
gen = ke.models.load_model("/content/gdrive/MyDrive/model/generator") #replace with location of model
disc = ke.models.load_model("/content/gdrive/MyDrive/model/discriminator") #replace with location of model


In [12]:
# prediction

def enhance(generator,img_lr:np.ndarray):
    img_shape = (1024,1024,3)
    img_hr_arr = np.zeros(img_shape)
    box_size = LR_shape[0]
    input_boxes = []
    for x in range(0,512,box_size):
        for y in range(0,512,box_size):
            lr_box = img_lr[x:x+box_size,y:y+box_size,:]
            input_boxes.append(lr_box)
    input_arr = np.array(input_boxes)
    output_arr = generator.predict(input_arr)
    n = 0
    for x in range(0,1024,s*box_size):
        for y in range(0,1024,s*box_size):
            img_hr_arr[x:x+s*box_size,y:y+s*box_size,:] = output_arr[n]
            n += 1
    img_hr_arr = 255*img_hr_arr
    img_hr = Image.fromarray(img_hr_arr.astype(np.uint8))
    return img_hr


In [13]:
img_num = '00000'
img_lr = get_img_arr("/content/gdrive/MyDrive/lr_imgs/"+img_num+"_lr.png") #replace with location of LR imgs
img_hr = enhance(gen,img_lr)
img_hr.save("test_enhanced_img.png")

8/8 [==============================] - 1s 60ms/step
